Carteira baseada no número de Graham sobre o valor intrínseco de uma ação, foca em pegar empresas que sejam lucrativas e baratas: empresas operando a P/L menor que 15 e com P/VPA menor que 1,5. Multiplicando 15 x 1,5 temos o número 22,5 que nos indica a fórmula de graham. Valor Intrínseco de uma ação = $\sqrt{22,5 \times \text{LPA} \times \text{VPA}}$.

Critérios:


- Ter Lucro por ação maior que zero ,isto é, a empresa não pode estar com prejuízo atualmente
- Ter Valor Patrimonial por ação positivo, ou seja, a empresa não pode ter mais passivos (obrigações a pagar) que ativos (bens ou direitos a receber)
- Ter volume médio de negociação diário de no mínimo R$250.000,00
- Ter lucro líquido medio positivo em todos os últimos 5 exercícios.


In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from IPython.display import display, HTML

def b_print(df , n=30 , clean=True): #beauty print :)
    
    # from IPython.display import display, HTML

    # if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
    #     df['prefixo'] = df['Papel'].astype(str).str[:4]
    #     df=df.drop_duplicates(subset='prefixo', keep='first')
    #     # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None


In [3]:
from DT_atualiza_settings import *
from DT_StatusInvest import SI
SI(mercado = 'Acoes' )

 ====== SI Acoes ===== 
 ====== Busca
 ====== Download 


In [4]:
import os
# Caminho do arquivo local
file_path = os.path.expanduser('~/GHub/Finance-playground/data/SI_Acoes.csv')
# URL para o arquivo online
file_url = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/refs/heads/main/data/SI_Acoes.csv'

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    funds = pd.read_csv(file_path,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    funds = pd.read_csv(file_url,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido da URL.")


Arquivo lido localmente.


In [5]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [6]:
fundsSI =  funds[ (funds[' LIQUIDEZ MEDIA DIARIA'] > 300000) &
                  (funds[' LPA'] > 0) & 
                  (funds[' VPA'] > 0) & 
                  (funds['CAGR LUCROS 5 ANOS'] > 0) ]
fundsSI

fundsSI['valor intrinseco'] = np.round(np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA']),2)
fundsSI['VALOR DE MERCADO (em B)'] = fundsSI[' VALOR DE MERCADO']  / 1e9
fundsSI['LIQUIDEZ MEDIA DIARIA (em M)'] = fundsSI[' LIQUIDEZ MEDIA DIARIA'] / 1e6
fundsSI['Delta (%)'] = np.round((fundsSI['valor intrinseco'] / fundsSI['PRECO'] -1)*100,2)
fundsSI["Rank"]   = fundsSI['Delta (%)'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

colunas_exibidas = ['TICKER','PRECO' , 'CAGR LUCROS 5 ANOS' ,'LIQUIDEZ MEDIA DIARIA (em M)',
                 ' VPA',' LPA' ,  'VALOR DE MERCADO (em B)',
                'valor intrinseco' , 'Delta (%)' ,'P/L', 'DIVIDA LIQUIDA / EBIT' ]

b_print(fundsSI[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
LIGT3,4.51,11.57,6.696793,15.09,6.16,1.680225,45.73,913.97,0.73,5.10
ALLD3,8.30,28.55,2.721039,17.82,3.72,0.787927,38.62,365.30,2.23,-0.32
BAZA3,76.88,31.27,0.336548,129.92,19.15,4.309763,236.60,207.75,4.01,NaN
SOMA3,5.95,80.87,99.716529,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
COGN3,3.21,41.43,87.761832,6.18,0.64,6.626297,9.43,193.77,4.98,3.26
TASA4,4.70,23.30,1.611882,9.29,0.89,0.669005,13.64,190.21,5.29,3.87
BRSR6,15.26,0.02,25.894157,27.14,3.11,6.528041,43.58,185.58,4.91,NaN
LOGG3,23.93,33.60,12.446378,43.62,4.37,2.102470,65.49,173.67,5.48,3.28
MTRE3,3.61,8.93,2.147587,9.35,0.46,0.381847,9.84,172.58,7.79,5.80
EUCA4,18.28,35.10,0.621642,30.27,3.19,1.784352,46.61,154.98,5.73,0.99


outros criterios adicionais

P/L > 0 e DIVIDA LIQUIDA / EBIT < 3

In [7]:
'''
P/L > 0 e DIVIDA LIQUIDA / EBIT < 3
'''
fundsSI.fillna(0,inplace=True) ## bancos tem NaN
b_print(fundsSI[(fundsSI['P/L'] > 0) & (fundsSI['DIVIDA LIQUIDA / EBIT'] < 3) ][colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
ALLD3,8.30,28.55,2.721039,17.82,3.72,0.787927,38.62,365.30,2.23,-0.32
BAZA3,76.88,31.27,0.336548,129.92,19.15,4.309763,236.60,207.75,4.01,0.00
SOMA3,5.95,80.87,99.716529,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
BRSR6,15.26,0.02,25.894157,27.14,3.11,6.528041,43.58,185.58,4.91,0.00
EUCA4,18.28,35.10,0.621642,30.27,3.19,1.784352,46.61,154.98,5.73,0.99
JHSF3,7.98,32.04,30.456363,9.23,1.90,5.436956,19.86,148.87,4.20,1.94
RECV3,11.12,57.65,40.907890,15.55,2.11,3.263410,27.17,144.33,5.26,1.75
MELK3,3.72,21.01,3.385617,5.32,0.57,0.767322,8.26,122.04,6.50,1.29
PETR4,30.41,14.07,1184.413285,32.81,6.01,404.672579,66.61,119.04,5.06,2.41
BMGB4,4.88,0.44,3.726125,7.67,0.66,2.923551,10.67,118.65,7.38,0.00


remove empresas repetidas, mantem primeira ocorrencia

In [8]:
fundsSI['prefixo'] = fundsSI['TICKER'].str[:4]
fundsSI.drop_duplicates(subset='prefixo', keep='first').head(20)
df_limpo = fundsSI.drop_duplicates(subset='prefixo', keep='first')
df_limpo = df_limpo.drop('prefixo', axis=1) 
b_print(df_limpo[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
LIGT3,4.51,11.57,6.696793,15.09,6.16,1.680225,45.73,913.97,0.73,5.10
ALLD3,8.30,28.55,2.721039,17.82,3.72,0.787927,38.62,365.30,2.23,-0.32
BAZA3,76.88,31.27,0.336548,129.92,19.15,4.309763,236.60,207.75,4.01,0.00
SOMA3,5.95,80.87,99.716529,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
COGN3,3.21,41.43,87.761832,6.18,0.64,6.626297,9.43,193.77,4.98,3.26
TASA4,4.70,23.30,1.611882,9.29,0.89,0.669005,13.64,190.21,5.29,3.87
BRSR6,15.26,0.02,25.894157,27.14,3.11,6.528041,43.58,185.58,4.91,0.00
LOGG3,23.93,33.60,12.446378,43.62,4.37,2.102470,65.49,173.67,5.48,3.28
MTRE3,3.61,8.93,2.147587,9.35,0.46,0.381847,9.84,172.58,7.79,5.80
EUCA4,18.28,35.10,0.621642,30.27,3.19,1.784352,46.61,154.98,5.73,0.99


================================================================================================

In [9]:
# Filtros de qualidade 
quality_filter = (
    (fundsSI[' LPA'] > 0) &
    (fundsSI[' VPA'] > 0) &
    (fundsSI['ROE'] > 0.08) &
    (fundsSI['DIVIDA LIQUIDA / EBIT'] < 3)
)
# Filtro de valor (Graham + margem)
value_filter = fundsSI['Delta (%)'] >= 25

# Catalisador operacional — crescimento de lucros
catalyst_lucro = fundsSI['CAGR LUCROS 5 ANOS'] > 0
# Catalisador de valuation — P/L deprimido
catalyst_pl = fundsSI['P/L'] < 12
# Catalisador patrimonial — P/VPA baixo
catalyst_pvp = fundsSI['P/VP'] < 1.2
# Pelo menos 1 catalisador ativo
fundsSI['Catalisador'] = (
    catalyst_lucro |
    catalyst_pl |
    catalyst_pvp
)
# Screening final 
screening = fundsSI[
    quality_filter &
    value_filter &
    fundsSI['Catalisador']
].copy()
# Score de position trade (simples e robusto)
screening['Score'] = (
    (screening['Delta (%)'] / 100) * 0.5 +
    (screening['CAGR LUCROS 5 ANOS'].clip(0, 20) / 100) * 0.3 +
    ((12 - screening['P/L']).clip(0, 12) / 12) * 0.2
)
#  Ordenação final
screening.sort_values('Score', ascending=False, inplace=True)
screening.reset_index(drop=True, inplace=True)
screening.index = screening.index + 1


cols = [
    'TICKER', 'PRECO',
    'valor intrinseco', 'Delta (%)',
    'CAGR LUCROS 5 ANOS',
    'P/L', 'P/VP',
    'ROE', 'DIVIDA LIQUIDA / EBIT',
    'Score'
]

b_print(screening[cols])

TICKER,PRECO,valor intrinseco,Delta (%),CAGR LUCROS 5 ANOS,P/L,P/VP,ROE,DIVIDA LIQUIDA / EBIT,Score
ALLD3,8.30,38.62,365.30,28.55,2.23,0.47,20.88,-0.32,2.049333
SOMA3,5.95,18.29,207.39,80.87,2.96,0.80,27.15,-0.45,1.247617
BAZA3,76.88,236.60,207.75,31.27,4.01,0.59,14.74,0.00,1.231917
BRSR6,15.26,43.58,185.58,0.02,4.91,0.56,11.45,0.00,1.046127
EUCA4,18.28,46.61,154.98,35.10,5.73,0.60,10.54,0.99,0.939400
JHSF3,7.98,19.86,148.87,32.04,4.20,0.86,20.57,1.94,0.934350
RECV3,11.12,27.17,144.33,57.65,5.26,0.72,13.59,1.75,0.893983
MELK3,3.72,8.26,122.04,21.01,6.50,0.70,10.75,1.29,0.761867
PETR4,30.41,66.61,119.04,14.07,5.06,0.93,18.33,2.41,0.753077
GMAT3,4.50,9.25,105.56,41.00,5.38,0.99,18.46,0.59,0.698133


Interpretação prática (position trade)

Delta alto + Score alto
→ desconto real com gatilho

Score alto, Delta médio
→ re-rating provável

Delta alto, Score baixo
→ possível value trap

================================================================================================

### outra estória de porque 22,5

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce. Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos. O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores. Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

# Stocks

In [10]:
import os
# Caminho do arquivo local /home/yair/GHub/Codigos-em-financas/data/SI_Stocks.csv
file_path = os.path.expanduser('/home/yair/GHub/Finance-playground/data/SI_Stocks.csv')
# URL para o arquivo online
file_url = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/refs/heads/main/data/SI_Stocks.csv'

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    funds = pd.read_csv(file_path,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    funds = pd.read_csv(file_url,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido da URL.")


Arquivo lido localmente.


In [11]:
funds

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
0,A,150.43,0.33,35.39,6.70,3.49,13.01,5.77,17.66,108.24,113.00,4.76,0.29,6.25,18.06,-5.35,2.25,18.92,9.86,2.45,0.52,0.48,0.56,4.75,3.77,NaN,22.47,4.25,-28.79,4.264697e+10
1,AA,44.93,0.22,10.15,1.83,0.73,6.00,2.98,8.91,30.38,36.29,5.91,0.36,0.90,6.13,-1.09,1.56,18.06,7.18,1.65,0.40,0.60,0.81,2.66,NaN,NaN,24.50,4.43,-0.02,1.163525e+10
2,AACG,1.07,NaN,-1.34,1.49,0.47,52.66,0.00,-9.12,NaN,NaN,NaN,-0.27,0.12,-1.02,-0.55,0.23,-111.44,-35.42,NaN,0.32,0.68,3.89,21.21,NaN,NaN,0.72,-0.80,-0.10,3.414520e+07
3,AACQ,9.99,NaN,-9.80,180.94,1.25,0.00,0.00,-383.09,NaN,NaN,NaN,-0.07,37.54,2094.26,-1.25,3.56,-1846.66,-12.73,NaN,0.01,0.12,0.03,NaN,NaN,NaN,0.06,-1.02,-0.01,9.047194e+08
4,AACQU,11.05,NaN,-10.84,200.14,1.38,0.00,0.00,-383.09,NaN,NaN,NaN,-0.07,41.52,2316.47,-1.38,3.56,-1846.66,-12.73,NaN,0.01,0.12,0.03,NaN,NaN,NaN,0.06,-1.02,-0.01,9.047194e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5289,ZUO,10.02,NaN,-20.82,8.36,1.80,67.42,-10.54,-16.34,-32.28,-28.30,3.99,-1.03,3.40,3.65,-8.80,2.62,-40.16,-8.63,-8.89,0.21,0.79,0.53,349.59,NaN,NaN,1.20,-0.48,0.40,1.540242e+09
5290,ZVO,0.09,NaN,-0.08,0.47,0.04,29.11,-18.74,-20.58,-0.09,0.67,0.76,-4.11,0.02,-1.38,-0.07,0.95,-593.02,-45.85,-541.17,0.08,0.92,2.23,-12.86,NaN,NaN,0.19,-1.15,0.00,3.078549e+06
5291,ZYME,26.26,NaN,-31.21,6.19,4.98,23.07,-13.23,-47.16,-111.28,-97.12,14.16,-0.79,14.72,7.93,-18.76,6.94,-19.82,-15.97,-7.86,0.81,0.19,0.34,20.90,NaN,NaN,4.25,-0.84,0.71,1.979734e+09
5292,ZYNE,1.30,NaN,-1.87,0.00,0.00,NaN,NaN,NaN,-8.75,4478.28,4487.03,-1.24,NaN,0.00,-0.06,3.43,-0.13,-0.09,-0.03,0.72,0.28,0.00,NaN,NaN,NaN,538.03,-0.70,4.43,7.012122e+07


In [12]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [13]:
fundsSI =  funds[ (funds[' LPA'] > 1) & 
                  (fundsSI['DIVIDA LIQUIDA / EBIT'] < 4) & 
                  (funds[' VPA'] > 0) 
                #   (funds['CAGR LUCROS 5 ANOS'] > 0) 
                  ]
b_print(fundsSI)

TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
AAON,88.37,0.45,71.96,8.36,4.70,118.76,9.90,33.67,244.76,256.90,12.14,0.41,24.23,14.17,-9.31,3.04,11.62,6.53,0.90,0.56,0.44,0.19,20.67,25.70,NaN,10.57,1.23,-0.72,7.214005e+09
AAPL,285.73,0.36,37.69,57.26,11.75,46.91,7.88,26.92,128.71,130.05,1.34,0.60,10.15,-238.88,-19.98,0.89,151.91,31.18,7.01,0.21,0.79,1.16,8.68,14.30,NaN,4.99,7.58,1.93,4.222047e+12
AAWW,102.48,NaN,0.02,957.44,438.20,25591.50,5305.22,3776.40,0.02,-4.35,-4.36,-252517.36,0.88,-63287.62,-526.68,0.96,4119089.58,1885228.67,2849874.03,0.46,0.54,499.21,16.10,9.75,NaN,0.11,4408.90,0.00,2.934330e+09
ABB,37.12,NaN,17.55,5.68,1.80,0.00,0.00,6.66,NaN,NaN,NaN,0.09,1.17,89.85,-3.35,1.04,32.33,10.23,NaN,0.32,0.68,1.54,3.17,2.26,NaN,6.54,2.11,-0.45,6.900207e+10
ABCB,76.59,0.52,13.14,1.30,0.19,0.00,32.51,95.87,38.75,16.24,-22.52,-0.76,12.60,NaN,-0.19,NaN,9.91,1.47,0.33,0.15,0.85,0.02,14.90,17.31,NaN,58.80,5.83,0.64,5.232254e+09
ABEO,4.64,NaN,3.05,1.54,1.02,98.45,-31.87,319.61,-30.62,-5.50,25.12,-1.26,9.76,1.27,-18.00,6.73,50.34,33.44,-4.66,0.66,0.34,0.10,NaN,NaN,NaN,3.02,1.52,-0.03,2.514481e+08
ABEV,2.61,2.43,0.02,2.33,1.50,51.55,16894754.57,16092445.36,0.02,0.01,0.00,-0.19,2537.73,32.43,-2.12,1.19,14773.65,9520.71,12218.27,0.64,0.36,0.00,2.06,-4.53,NaN,1.12,165.51,0.00,4.069634e+10
ABG,236.32,NaN,8.19,1.18,0.39,16.99,1.34,3.15,19.17,42.92,23.76,1.47,0.26,-184.51,-0.54,0.99,14.46,4.76,0.51,0.33,0.67,1.51,18.98,18.47,NaN,199.48,28.85,0.14,4.594203e+09
ABM,43.92,0.60,15.88,1.47,0.51,25.64,0.98,1.95,31.75,48.99,17.24,0.80,0.31,4.21,-0.81,1.49,9.25,3.21,0.60,0.35,0.65,1.65,5.16,-8.57,NaN,29.90,2.77,0.22,2.689393e+09
ABMD,381.02,NaN,64.40,11.16,10.13,61.12,18.85,24.84,84.86,80.81,-4.05,-0.53,16.00,18.50,-27.11,7.93,17.32,15.72,8.32,0.91,0.09,0.63,18.30,21.24,NaN,34.15,5.92,0.82,1.718065e+10


In [14]:
fundsSI['valor intrinseco'] = np.round(np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA']),2)
fundsSI['VALOR DE MERCADO (em B)'] = fundsSI[' VALOR DE MERCADO']  / 1e9
# fundsSI['LIQUIDEZ MEDIA DIARIA (em M)'] = fundsSI[' LIQUIDEZ MEDIA DIARIA'] / 1e6
fundsSI['Delta (%)'] = np.round((fundsSI['valor intrinseco'] / fundsSI['PRECO'] -1)*100,2)
fundsSI["Rank"]   = fundsSI['Delta (%)'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

colunas_exibidas = ['TICKER','PRECO' , 'CAGR LUCROS 5 ANOS' ,
                 ' VPA',' LPA' ,  'VALOR DE MERCADO (em B)',
                'valor intrinseco' , 'Delta (%)' ,'P/L', 'DIVIDA LIQUIDA / EBIT' ]

b_print(fundsSI[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
AFYA,15.20,21.91,8253.14,1313.56,1.378605,15618.01,102650.07,0.01,2.23
ABEV,2.61,-4.53,1.12,165.51,40.696339,64.58,2374.33,0.02,0.00
AEO,23.93,11.48,9.12,473.09,4.052277,311.57,1202.01,0.05,0.00
ACIC.WS,1.68,NaN,6.71,1.75,0.591035,16.25,867.26,0.96,-5.72
AFGE,16.99,-0.22,56.71,9.57,10.958399,110.50,550.38,1.78,4.59
ACGLO,21.07,22.01,65.41,11.19,33.586411,128.33,509.07,1.88,2.25
AFGC,18.92,-0.22,56.71,9.57,10.958399,110.50,484.04,1.98,4.59
AFGD,20.91,-0.22,56.71,9.57,10.958399,110.50,428.46,2.19,4.59
ACGLP,24.99,22.01,65.41,11.19,33.586411,128.33,413.53,2.23,2.25
AFGB,21.87,-0.22,56.71,9.57,10.958399,110.50,405.26,2.29,4.59
